<img src='img/fon.png' width=200>
<center>
    <br/>
    Otkrivanje zakonitosti u podacima 2019-2020
    <br/>
    Fakultet Organizacionih Nauka, 2020
    <br/>
    <h1>D2. Domaći zadatak 2</h1>
</center>

# Zahtevi domaćeg zadatka


0. Učitati podatke o vinima (wine_quality.csv), sa ciljem prepoznavanja "top_quality" klase.

1. Kreirati minimalno 3 modela za klasifikaciju (sa default parametrima)
    - proceniti grešku generalizacije za sve modele, na osnovu nekoliko metrika;
    - prikazati i standardnu devijaciju procene greške (od 10 kros-validacionih uzoraka)
    - odgovoriti da li su modeli pretrenirani;
    - prodiskutovati koja metrika je najadekvatnija za ovaj problem, i koji tip grešaka je skuplji;
    - modifikovati granicu odlučivanja, tako da model pravi više jeftinijih grešaka;
    - odgovoriti koji atributi su najbitniji za naučene modele (gde je moguće videti)

2. Optimizovati minimalno 2 parametra kod najboljeg modela (iz koraka 1)
    - ispitati najmanje 100 različitih vrednosti parametara
    - uporediti kvalitet optimizovanog i neoptimizovanog modela

3. Napraviti 2 razlicita ansambl modela, bazirana na modelima iz koraka 2
    - optimizovati parametre ansambl modela
    - uporediti kvalitet ansambl i osnovnih modela

In [1]:
%matplotlib inline

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

## Učitavanje podataka

0. Učitati podatke o vinima (wine_quality.csv), sa ciljem prepoznavanja "top_quality" klase.

In [2]:
df = pd.read_csv('data/wine_quality.csv')
df.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,top_quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,0
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,0
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,0
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,0
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,0


In [4]:
df.top_quality.value_counts(normalize=True)

0    0.783585
1    0.216415
Name: top_quality, dtype: float64

In [8]:
df.shape

(4898, 12)

## Uvidi o potrebnom preprocesiranju

Svi atributi su numerički, tako da nema potrebe za enkodovanjem. Na različitim su skalama, pa ih moramo normalizovati. Vrednosti target klase su u odnosu 4:1, pa je potrebno balansirati podatke.

## Kreiranje modela

### Otklanjanje izuzetaka

In [5]:
from scipy import stats

df_no_out = df.loc[(np.abs(stats.zscore(df)) < 3).all(axis=1), :]

In [7]:
print(f'{str(df.shape[0] - df_no_out.shape[0])} outliers have been removed. Percentage: {str((df.shape[0] - df_no_out.shape[0]) / df.shape[0])}')

396 outliers have been removed. Percentage: 0.08084932625561454


### Podela na trening i test podatke

In [11]:
X = df_no_out.drop('top_quality', axis=1)
y = df_no_out['top_quality']

In [12]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

In [16]:
print(f'Velicina treninga: {str(X_train.shape[0])}. Velicina testa: {str(X_test.shape[0])}')

Velicina treninga: 3151. Velicina testa: 1351


### Skaliranje podataka

Korišćen je MaxMinScaler koji je "naučen" na treningu, a onda samo iskorišćen za transformaciju testa.

In [18]:
from sklearn.preprocessing import MinMaxScaler

sc = MinMaxScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)
X_train

array([[0.79591837, 0.38      , 0.49275362, ..., 0.45882353, 0.47540984,
        0.32758621],
       [0.28571429, 0.46      , 0.47826087, ..., 0.45882353, 0.62295082,
        0.36206897],
       [0.79591837, 0.14      , 0.56521739, ..., 0.34117647, 0.27868852,
        0.36206897],
       ...,
       [0.51020408, 0.34      , 0.42028986, ..., 0.25882353, 0.2295082 ,
        0.56896552],
       [0.42857143, 0.6       , 0.49275362, ..., 0.41176471, 0.8852459 ,
        0.27586207],
       [0.32653061, 0.2       , 0.44927536, ..., 0.64705882, 0.59016393,
        0.46551724]])

### Balansiranje podataka

In [20]:
from imblearn.under_sampling import RandomUnderSampler

under_sampler = RandomUnderSampler(random_state=0)
X_train, y_train = under_sampler.fit_resample(X_train, y_train)


# from imblearn.over_sampling import SMOTE

# smote_over_sampler = SMOTE(random_state=0)
# X_train, y_train = smote_over_sampler.fit_sample(X_train, y_train.ravel())

### Inicijalni modeli

1. Kreirati minimalno 3 modela za klasifikaciju (sa default parametrima)

In [22]:
from sklearn.linear_model import LogisticRegression

model_lr = LogisticRegression()
model_lr.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [23]:
from sklearn.naive_bayes import GaussianNB

model_nb = GaussianNB()
model_nb.fit(X_train, y_train)

GaussianNB(priors=None, var_smoothing=1e-09)

In [24]:
from sklearn.tree import DecisionTreeClassifier

model_tree = DecisionTreeClassifier()
model_tree.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

In [25]:
from sklearn.neighbors import KNeighborsClassifier

model_knn = KNeighborsClassifier()
model_knn.fit(X_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')

In [31]:
from sklearn.ensemble import RandomForestClassifier

model_rff = RandomForestClassifier()
model_rff.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

#### Evaluacija 

    - proceniti grešku generalizacije za sve modele, na osnovu nekoliko metrika;

In [32]:
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, roc_auc_score
from prettytable import PrettyTable

In [33]:
def evaluate(models, scores, X_fun=X_train, y_fun=y_train):
    for model in models:
        print(str(model).split('(')[0])
        y_pred = model.predict(X_fun)
        t = PrettyTable(['Mera evaluacije', 'Rezultat'])
        for score in scores:
            s = score(y_fun, y_pred)
            t.add_row([str(score).split()[1], s])
        print(t)
        print('\n')

In [34]:
models = [model_lr, model_nb, model_tree, model_knn, model_rff]
scores = [accuracy_score, precision_score, recall_score, roc_auc_score]

evaluate(models, scores, X_train, y_train)

LogisticRegression
+-----------------+--------------------+
| Mera evaluacije |      Rezultat      |
+-----------------+--------------------+
|  accuracy_score | 0.7391608391608392 |
| precision_score | 0.7261904761904762 |
|   recall_score  | 0.7678321678321678 |
|  roc_auc_score  | 0.7391608391608392 |
+-----------------+--------------------+


GaussianNB
+-----------------+--------------------+
| Mera evaluacije |      Rezultat      |
+-----------------+--------------------+
|  accuracy_score | 0.7097902097902098 |
| precision_score | 0.6847290640394089 |
|   recall_score  | 0.7776223776223776 |
|  roc_auc_score  | 0.7097902097902098 |
+-----------------+--------------------+


DecisionTreeClassifier
+-----------------+----------+
| Mera evaluacije | Rezultat |
+-----------------+----------+
|  accuracy_score |   1.0    |
| precision_score |   1.0    |
|   recall_score  |   1.0    |
|  roc_auc_score  |   1.0    |
+-----------------+----------+


KNeighborsClassifier
+---------------

In [35]:
evaluate(models, scores, X_test, y_test)

LogisticRegression
+-----------------+---------------------+
| Mera evaluacije |       Rezultat      |
+-----------------+---------------------+
|  accuracy_score |  0.7172464840858623 |
| precision_score | 0.42803030303030304 |
|   recall_score  |  0.738562091503268  |
|  roc_auc_score  |  0.7247834380961315 |
+-----------------+---------------------+


GaussianNB
+-----------------+---------------------+
| Mera evaluacije |       Rezultat      |
+-----------------+---------------------+
|  accuracy_score |  0.6839378238341969 |
| precision_score | 0.39658119658119656 |
|   recall_score  |  0.7581699346405228 |
|  roc_auc_score  |  0.710185445789161  |
+-----------------+---------------------+


DecisionTreeClassifier
+-----------------+--------------------+
| Mera evaluacije |      Rezultat      |
+-----------------+--------------------+
|  accuracy_score | 0.7394522575869726 |
| precision_score | 0.4555984555984556 |
|   recall_score  | 0.7712418300653595 |
|  roc_auc_score  | 0.750

### Evaluacija unakrsnom kros-validacijom

    - prikazati i standardnu devijaciju procene greške (od 10 kros-validacionih uzoraka)

In [40]:
from sklearn.model_selection import cross_val_score

for model, label in zip([model_lr, model_nb, model_tree, model_knn, model_rff], ['Logisticka regresija', 'Naive bayes','Stablo odlucivanja', 'K-NN', 'Random Forest']):
    scores = cross_val_score(model, X, y, cv=10, scoring='accuracy')
    print('[%s] Tacnost: %0.2f +/- %0.2f' % (label, scores.mean(), scores.std()))

[Logisticka regresija] Tacnost: 0.78 +/- 0.03
[Naive bayes] Tacnost: 0.75 +/- 0.05
[Stablo odlucivanja] Tacnost: 0.74 +/- 0.04
[K-NN] Tacnost: 0.74 +/- 0.03
[Random Forest] Tacnost: 0.79 +/- 0.02


    - odgovoriti da li su modeli pretrenirani;

Možemo reći da su NB i DT model pretrenirani.

    - prodiskutovati koja metrika je najadekvatnija za ovaj problem, i koji tip grešaka je skuplji;
    
Recall, objasniti ukratko.

    - modifikovati granicu odlučivanja, tako da model pravi više jeftinijih grešaka;
    

    - odgovoriti koji atributi su najbitniji za naučene modele (gde je moguće videti)

### Optimizacija metaparametara

2. Optimizovati minimalno 2 parametra kod najboljeg modela (iz koraka 1)
    - ispitati najmanje 100 različitih vrednosti parametara
    - uporediti kvalitet optimizovanog i neoptimizovanog modela

In [41]:
from sklearn.model_selection import GridSearchCV, cross_val_score

params = {
    'criterion': ['entropy', 'gini'],
    'min_samples_leaf': range(1,30,3),
    'max_depth': range(2,16,2)
}

grid_searh = GridSearchCV(DecisionTreeClassifier(), params, cv=10, scoring='recall')
grid_searh.fit(X_train, y_train)

grid_searh.best_estimator_

/Users/dimitrijemilenkovic/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:813: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=8,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

In [42]:
model_tree_param = grid_searh.best_estimator_

In [43]:
evaluate([model_tree_param, model_tree], [accuracy_score, roc_auc_score, recall_score, precision_score], X_test, y_test)

DecisionTreeClassifier
+-----------------+--------------------+
| Mera evaluacije |      Rezultat      |
+-----------------+--------------------+
|  accuracy_score | 0.6802368615840119 |
|  roc_auc_score  | 0.7309034618632142 |
|   recall_score  | 0.8235294117647058 |
| precision_score |        0.4         |
+-----------------+--------------------+


DecisionTreeClassifier
+-----------------+--------------------+
| Mera evaluacije |      Rezultat      |
+-----------------+--------------------+
|  accuracy_score | 0.7394522575869726 |
|  roc_auc_score  | 0.7506926853676079 |
|   recall_score  | 0.7712418300653595 |
| precision_score | 0.4555984555984556 |
+-----------------+--------------------+




### Kreiranje ansambl modela

3. Napraviti 2 razlicita ansambl modela, bazirana na modelima iz koraka 2
    - optimizovati parametre ansambl modela
    - uporediti kvalitet ansambl i osnovnih modela